In [1]:
from network import Net
from optimizers import Optimizer
from tensorflow.keras.datasets import mnist
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

def load_mnist():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = np.reshape(x_train, [-1, 28*28])
    x_test = np.reshape(x_test, [-1, 28*28])
    y_train = tf.keras.utils.to_categorical(y_train, 10)
    y_test = tf.keras.utils.to_categorical(y_test, 10)
    x_train = x_train/np.max(x_train)
    x_test = x_test/np.max(x_test)
    return x_train, y_train, x_test, y_test

def run_tf_model(x_train, y_train, x_test, y_test):
    model = Sequential()
    model.add(Dense(256, activation='relu', input_shape=(784,)))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    print("\nFitting Model...")
    model.fit(x_train, y_train, batch_size=32, epochs=5, validation_split=0.2)
    print("\nEvaluating Model...")
    model.evaluate(x_test, y_test)


In [2]:
opt = Optimizer()

x_train, y_train, x_test, y_test= load_mnist()
pca = PCA(10)
x_train = pca.fit_transform(x_train)
x_test = pca.transform(x_test)

print("Starting Distribute Fractional")
epoch = 5
model = Net(x_train, y_train, [10, 10], batch_size=64)

Starting Distribute Fractional


In [3]:
x_test.shape[0]

10000

In [4]:
model.eval(model.w_flatten, x_test, y_test)

0
64
128
192
256
320
384
448
512
576
640
704
768
832
896
960
1024
1088
1152
1216
1280
1344
1408
1472
1536
1600
1664
1728
1792
1856
1920
1984
2048
2112
2176
2240
2304
2368
2432
2496
2560
2624
2688
2752
2816
2880
2944
3008
3072
3136
3200
3264
3328
3392
3456
3520
3584
3648
3712
3776
3840
3904
3968
4032
4096
4160
4224
4288
4352
4416
4480
4544
4608
4672
4736
4800
4864
4928
4992
5056
5120
5184
5248
5312
5376
5440
5504
5568
5632
5696
5760
5824
5888
5952
6016
6080
6144
6208
6272
6336
6400
6464
6528
6592
6656
6720
6784
6848
6912
6976
7040
7104
7168
7232
7296
7360
7424
7488
7552
7616
7680
7744
7808
7872
7936
8000
8064
8128
8192
8256
8320
8384
8448
8512
8576
8640
8704
8768
8832
8896
8960
9024
9088
9152
9216
9280
9344
9408
9472
9536
9600
9664
9728
9792
9856
9920


2.266426349547296

In [12]:
opt = Optimizer()

x_train, y_train, x_test, y_test= load_mnist()
pca = PCA(10)
x_train = pca.fit_transform(x_train)

print("Starting Distribute Fractional")
epoch = 5
model = Net(x_train, y_train, [10, 10], batch_size=64)
hist_dist = []
for ep in range(epoch):
    print("EPOCH: ", ep)
    for b in range(int(x_test.shape[0]/model.batch_size)):
        print(f"Distribute, EPOCH: {ep}, Batch step: {b}")
        opt.dist_frac_optimizer(model.categorical_cross_entropy, model.w_flatten,  lr=0.03, alpha1=0.9, alpha2=1.1, max_iter=10, N=10)
        hist_dist.append(model.categorical_cross_entropy(model.w_flatten))

        model.batch_counter += model.batch_size

Starting Distribute Fractional
EPOCH:  0
Distribute, EPOCH: 0, Batch step: 0
Distribute, EPOCH: 0, Batch step: 1
Distribute, EPOCH: 0, Batch step: 2


KeyboardInterrupt: 

In [5]:
model = Net(x_train, y_train, [10, 10], batch_size=64)
opt = Optimizer()

In [6]:
epoch = 5
model = Net(x_train, y_train, [10, 10], batch_size=8)
hist_int = []
for ep in range(epoch):
    print("EPOCH: ", ep)
    for b in range(int(x_test.shape[0]/model.batch_size)):
        print("Batch step: ", b)
        opt.optimizer(model.categorical_cross_entropy, model.w_flatten, lr=0.03, max_iter=10)
        hist_int.append(model.categorical_cross_entropy(model.w_flatten))

        model.batch_counter += model.batch_size

EPOCH:  0
Batch step:  0


TypeError: categorical_cross_entropy() missing 1 required positional argument: 'w'

In [ ]:
epoch = 5
model = Net(x_train, y_train, [10, 10], batch_size=64)
hist_frac = []
for ep in range(epoch):
    print("EPOCH: ", ep)
    for b in range(int(x_test.shape[0]/model.batch_size)):
        print("Batch step: ", b)
        opt.frac_optimizer(model.categorical_cross_entropy, model.w_flatten,  lr=0.03, alpha=0.9, , max_iter=10)
        hist_frac.append(model.categorical_cross_entropy())

        model.batch_counter += model.batch_size

In [ ]:
epoch = 5
model = Net(x_train, y_train, [10, 10], batch_size=64)
hist_multi = []
for ep in range(epoch):
    print("EPOCH: ", ep)
    for b in range(int(x_test.shape[0]/model.batch_size)):
        print("Batch step: ", b)
        opt.multi_frac_optimizer(model.categorical_cross_entropy, model.w_flatten,  lr=0.03, alpha1=1.1, alpha2=0.9, return_history=True)
        hist_multi.append(model.categorical_cross_entropy())
        
        model.batch_counter += model.batch_size

In [ ]:
epoch = 5
model = Net(x_train, y_train, [10, 10], batch_size=64)
hist_dist = []
for ep in range(epoch):
    print("EPOCH: ", ep)
    for b in range(int(x_test.shape[0]/model.batch_size)):
        print("Batch step: ", b)
        x, history_dist_frac = opt.dist_frac_optimizer(model.categorical_cross_entropy, model.w_flatten,  lr=0.03, alpha1=0.1, alpha2=1.1, return_history=True)
        hist_dist.append(model.categorical_cross_entropy())

        model.batch_counter += model.batch_size

In [17]:
%time opt.optimizer(model.categorical_cross_entropy, model.w_flatten, lr=0.03, max_iter=10)

CPU times: total: 312 ms
Wall time: 322 ms


array([-0.02333728,  0.14225839, -0.04789694, -0.0322301 ,  0.05181059,
        0.03096294,  0.02266131,  0.12780108,  0.10626587,  0.09338487,
        0.06126034, -0.01642846, -0.05577342,  0.05568581,  0.1072452 ,
        0.10708783,  0.0338406 , -0.19853292,  0.09752564,  0.05151447,
        0.03661423, -0.08228161,  0.08787268,  0.04075819, -0.01195115,
        0.07168787, -0.02799571,  0.11263465,  0.01563215,  0.1108363 ,
       -0.07311283, -0.04667382, -0.00859593,  0.09208404, -0.10171706,
        0.05033377, -0.06934804, -0.02355108,  0.11061242, -0.05596373,
       -0.19924411, -0.05536904,  0.04049719, -0.11546708, -0.0569809 ,
       -0.00683506,  0.00085073, -0.117137  , -0.18462774,  0.13386428,
       -0.05943423, -0.19146689,  0.11058676, -0.29283497,  0.07732571,
       -0.10613036, -0.06131293, -0.04945274,  0.05641555,  0.16467062,
        0.08808146, -0.01398465, -0.05645471,  0.06275319, -0.01111421,
       -0.00350839,  0.11640827,  0.01401406,  0.07886435,  0.00

In [18]:
%time opt.frac_optimizer(model.categorical_cross_entropy, model.w_flatten, lr=0.03, max_iter=10, alpha=0.95)

CPU times: total: 328 ms
Wall time: 343 ms


array([-0.02147662,  0.13776543, -0.04026609, -0.03118188,  0.04837025,
        0.03425193,  0.01886791,  0.12912531,  0.10260507,  0.08890155,
        0.05710685, -0.01550622, -0.05626617,  0.05367513,  0.11064896,
        0.11150684,  0.03005962, -0.19556602,  0.09349893,  0.04838228,
        0.0379145 , -0.08186026,  0.08960573,  0.03872789, -0.01434859,
        0.07228507, -0.02786948,  0.11117889,  0.01740664,  0.11112681,
       -0.07539317, -0.04563318, -0.01075019,  0.0890507 , -0.09918355,
        0.05113172, -0.06743553, -0.02286299,  0.10753079, -0.0524003 ,
       -0.19595149, -0.05506979,  0.03666558, -0.11259128, -0.05954059,
       -0.00610493, -0.0009785 , -0.11555228, -0.18213994,  0.13126947,
       -0.05700191, -0.19086002,  0.10836925, -0.29221048,  0.07569965,
       -0.10615903, -0.0600447 , -0.05059827,  0.05803064,  0.16377056,
        0.08615781, -0.01425839, -0.05772636,  0.05959656, -0.01265029,
       -0.00526317,  0.11421363,  0.01510783,  0.0822431 ,  0.00

In [19]:
%time opt.multi_frac_optimizer(model.categorical_cross_entropy, model.w_flatten, lr=0.03, max_iter=10, alpha1=0.9, alpha2=1.1)

CPU times: total: 672 ms
Wall time: 673 ms


array([-0.02691121,  0.14725103, -0.06169464, -0.03338447,  0.05557795,
        0.02483999,  0.02817968,  0.12342567,  0.11049467,  0.09858546,
        0.06829589, -0.01795524, -0.05485006,  0.05863298,  0.10060001,
        0.09939321,  0.04011396, -0.20190202,  0.10342511,  0.05621972,
        0.03413744, -0.08313344,  0.08498688,  0.043609  , -0.00816984,
        0.07054053, -0.02832583,  0.1142217 ,  0.01246205,  0.1101573 ,
       -0.06909521, -0.04767218, -0.00513117,  0.09615786, -0.10557214,
        0.04906969, -0.07164314, -0.02378531,  0.11532672, -0.0606858 ,
       -0.20376275, -0.05546472,  0.04658236, -0.11890715, -0.05293693,
       -0.00788953,  0.00355412, -0.11875388, -0.18787884,  0.13708327,
       -0.06308343, -0.19213949,  0.11399865, -0.29360965,  0.08008124,
       -0.10596645, -0.06327494, -0.04724663,  0.05408353,  0.16594181,
        0.09093137, -0.01352002, -0.05442286,  0.06754926, -0.00846563,
       -0.00069159,  0.11943291,  0.01189809,  0.07370664,  0.00

In [20]:
%time opt.dist_frac_optimizer(model.categorical_cross_entropy, model.w_flatten, lr=0.03, max_iter=10, alpha1=0.9, alpha2=1.1, N=10)

CPU times: total: 3.61 s
Wall time: 3.63 s


array([-0.05037425,  0.38291342, -0.45154315, -0.06655738,  0.23835693,
       -0.07624213,  0.19563619,  0.16537915,  0.30630399,  0.31191481,
        0.21640397, -0.05555989, -0.03700483,  0.12301176,  0.01027797,
       -0.07385754,  0.17794679, -0.36431701,  0.27864968,  0.19801932,
        0.01236693, -0.09142234,  0.03395458,  0.13159884,  0.08236803,
        0.06393332, -0.01641705,  0.19315974, -0.02688114,  0.12601803,
       -0.04751909, -0.11295669,  0.07006825,  0.18982314, -0.18776753,
        0.01870095, -0.18777985, -0.08628216,  0.23783317, -0.19066579,
       -0.3308488 , -0.09898938,  0.19976541, -0.23607001,  0.02977464,
       -0.03939519,  0.07901114, -0.21843533, -0.29577021,  0.2467224 ,
       -0.14272266, -0.22784966,  0.18906491, -0.3058541 ,  0.12737471,
       -0.12105304, -0.09931508, -0.04218427,  0.00979996,  0.20395627,
        0.14727423, -0.00708635, -0.00596789,  0.16979906,  0.03978505,
        0.06374561,  0.21794286,  0.01802765, -0.08832608, -0.05